# Coins

## Statement of the problem

Every currency system has denominations of currency. For example, for coins in the United States, we have pennies, nickels, dimes, and quarters, which are worth 1 cent, 5 cents, 10 cents, and 25 cents respectively. 

Consider the following scenario. You know that you will need to pay between 0 and 1 dollar, and want to make exact change. What is the minimum number of coins that you need? The answer is 3 quarters, 2 dimes, and 5 pennies, so 10 coins in total.

The problem is this: find a denomination that minimizes this number.

## Formalization

We start with the notion of a denomination. To keep things separate, we will create a separate class for a single denomination system.

In [1]:
from math import floor, ceil

class DenominationSet:
    def __init__(self, denominations : list[int]):
        self.denominations = denominations
    
    def __getitem__(self, i) -> int:
        return self.denominations[i]
    
    def __len__(self) -> int:
        return len(self.denominations)

    def __str__(self) -> str:
        return str(self.denominations)
    
    def __repr__(self):
        return str(self)
    
    def __hash__(self):
        return hash(str(self.denominations))
    
    def __eq__(self, obj):
        return self.denominations == obj.denominations

For example, here is the United States coins. (We require that they are ordered greatest to least)

In [2]:
us_coins = DenominationSet([25, 10, 5, 1])

assert us_coins[0] == 25
assert us_coins[1] == 10
assert us_coins[2] == 5
assert us_coins[3] == 1

Next, we have a set of coins.

In [3]:
class CoinSet:
    def __init__(self, counts : list[int], denomination_set: DenominationSet):
        self.denomination_set = denomination_set
        self.counts = counts
    
    def __getitem__(self, i) -> int:
        return self.counts[i]
    
    def __str__(self):
        to_return = ""
        for i in range(len(self.denomination_set)):
            to_return += f"({self[i]}*{self.denomination_set[i]}), "
        return f"[{to_return[:-2]}]"
    
    def __repr__(self):
        return str(self)
    
    def __hash__(self):
        return hash((self.denomination_set, str(self.counts)))
    
    def __eq__(self, obj):
        return self.counts == obj.counts and self.denomination_set == obj.denomination_set
    
# 2 quarters, a dime, a nickel, and 3 pennies
my_coins = CoinSet([2, 1, 1, 3], us_coins)

assert my_coins[0] == 2
assert my_coins[1] == 1
assert my_coins[2] == 1
assert my_coins[3] == 3
assert my_coins.denomination_set == us_coins

print(my_coins)

[(2*25), (1*10), (1*5), (3*1)]


## Basic Operations

In [4]:
def get_total(coins: CoinSet) -> int:
    count = 0
    for i in range(len(coins.denomination_set)):
        count += coins[i] * coins.denomination_set[i]
    return count

print(get_total(my_coins))

def get_number_of_coins(coins: CoinSet) -> int:
    count = 0
    for i in range(len(coins.denomination_set)):
        count+=coins[i]
    return count

print(get_number_of_coins(my_coins))

68
7


In [5]:
#I think this is correct
def get_minimum_required_coins_for_cost(cost : int, denomination_set: DenominationSet) -> CoinSet:
    current_cost = cost
    coins = []
    for i in range(len(denomination_set)):
        new_coins = floor(current_cost / denomination_set[i])
        current_cost-=new_coins*denomination_set[i]
        coins.append(new_coins)
    return CoinSet(coins, denomination_set)

print(get_minimum_required_coins_for_cost(99, us_coins))

[(3*25), (2*10), (0*5), (4*1)]


## Naive Attempt

In [6]:
# Get all denominations where the maximum coin is less than the specified maximum
def get_all_possible_denominations(maximum : int) -> list[DenominationSet]:
    denominations = []
    for i in range(((2**(maximum-1)))):
        denominations.append(DenominationSet([maximum-a for a in range(maximum) if (((i >> a) & 1) == 1)]+[1]))
    return denominations

for i in get_all_possible_denominations(5):
    print(i)

[1]
[5, 1]
[4, 1]
[5, 4, 1]
[3, 1]
[5, 3, 1]
[4, 3, 1]
[5, 4, 3, 1]
[2, 1]
[5, 2, 1]
[4, 2, 1]
[5, 4, 2, 1]
[3, 2, 1]
[5, 3, 2, 1]
[4, 3, 2, 1]
[5, 4, 3, 2, 1]


In [7]:
from itertools import permutations
import tabulate

class SubsequenceDecay:
    '''
    Decay single.
    '''
    def decay(input):
        for i_n in range(len(input)):
            i = len(input) -1 - i_n
            for j_n in range(len(input)-i):
                j = j_n + i
                if j <= i:
                    continue
                current = input[i]
                next = input[j]
                if next > current + 1:
                    input[i] = current + 1
                    input[j] = next - 1
                    return input
        return input
    
    def get_initial_state(i, r):
        return [1 for i in range(i)] + [r]

    def decay_fully(i, r):
        current = SubsequenceDecay.get_initial_state(i, r)
        yield current.copy()
        previous = None
        while current != previous:
            previous = current.copy()
            current = SubsequenceDecay.decay(current)
            if current == previous:
                break
            yield current.copy()

def get_all_permutations_summing_to_x(x : int, length: int):
    if (x - length + 1 <= 0):
        return
    for ordered_sum in SubsequenceDecay.decay_fully(length - 1, x - length + 1):
        for permutation in permutations(ordered_sum):
            yield permutation
    
def can_coin_set_reach_total(coin_set : CoinSet, total : int):
    current = total
    for i in range(len(coin_set.denomination_set)):
        if (current == 0):
            return True
        amount_taken = min(floor(current/coin_set.denomination_set[i]), coin_set[i])*coin_set.denomination_set[i]
        current -= amount_taken
    return current == 0

USE_QUICK_COVERING_ANALYSIS = True
USE_QUICK_COVERING_GENERATION = True # Does not return all covering coin sets!!!

def evaluate_denomination_naive(maximum: int, d : DenominationSet, allow_vestigal = True, stop_search_at = None) -> list[CoinSet]:
    #print(f"================ Evaluating {d} ================")
    if USE_QUICK_COVERING_GENERATION:
        return [get_minimum_covering_coin_set(d, maximum, allow_vestigal = allow_vestigal)]
    else:
        for x in range(maximum*2):
            if stop_search_at != None and x > stop_search_at:
                return None
            #print(f"===== X = {x} =====")
            solutions = []
            done = False
            for a in get_all_permutations_summing_to_x(x, len(d)):
                coins = CoinSet( [e-1 for e in a] if allow_vestigal else a, d)
                if USE_QUICK_COVERING_ANALYSIS:
                    if is_covering_coin_set(coins, maximum):
                        done = True
                        solutions.append(coins)
                else:
                    can_reach_all = True
                    for i in range(maximum):
                        if not can_coin_set_reach_total(coins, i):
                            can_reach_all = False
                            break
                    if can_reach_all:
                        done = True
                        solutions.append(coins)
            if done == True:

                return list(set(solutions))

def evaluate(maximum: int, allow_vestigal = True, quick = False) -> list[dict]:
    data = []
    minimum = maximum*2
    for d in get_all_possible_denominations(maximum):
        result = evaluate_denomination_naive(maximum, d, allow_vestigal = allow_vestigal, stop_search_at = None if not quick else minimum)
        if result == None:
            continue
        score = get_number_of_coins(result[0])
        if (score < minimum and quick):
            minimum = score
            data = [] #All of the data is for greater values
        elif (score > minimum and quick):
            continue
        data.append({
            'denomination': d,
            'score': get_number_of_coins(result[0]),
            'number': len(result),
            'results': result
        })
    data.sort(key= lambda a: a['score'])
    
    return data

Here is the widget for evaluating this!

In [8]:
import ipywidgets as widgets
import time
from IPython.display import HTML, display

button = widgets.Button(description="Compute")
maximum_input = widgets.BoundedIntText(
    value=10,
    min=0,
    max=100,
    step=1,
    description='Maximum:',
    disabled=False
)
allow_vestigal_input = widgets.Checkbox(
    value=False,
    description='Allow Vestigal?',
    disabled=False,
    indent=False
)
quick_input = widgets.Checkbox(
    value=True,
    description='Quick',
    disabled=False,
    indent=False
)

def run_evaluation(a):
    start_time = time.time()
    table = tabulate.tabulate(evaluate(maximum_input.value, allow_vestigal=allow_vestigal_input.value, quick=quick_input.value)[:100], tablefmt='html', headers='keys')
    display(table)
    print(f"Completed in {time.time()-start_time} seconds.")

button.on_click(run_evaluation)
display(maximum_input)
display(allow_vestigal_input)
display(quick_input)
display(button)


BoundedIntText(value=10, description='Maximum:')

Checkbox(value=False, description='Allow Vestigal?', indent=False)

Checkbox(value=True, description='Quick', indent=False)

Button(description='Compute', style=ButtonStyle())

In [9]:
def get_reaching_coin_set(coin_set : CoinSet, total : int):
    current = total
    reaching_coin_set = []
    for i in range(len(coin_set.denomination_set)):
        if (current == 0):
            reaching_coin_set.append(0)
            continue
        coins_taken = min(floor(current/coin_set.denomination_set[i]), coin_set[i])
        amount_taken = coins_taken*coin_set.denomination_set[i]
        reaching_coin_set.append(coins_taken)
        current -= amount_taken
    return CoinSet(reaching_coin_set, coin_set.denomination_set)

def string_to_coin_set(string, denomination):
    coin_values = string.split(",")
    return CoinSet([int(a) for a in coin_values], denomination)
    
def string_to_denomination(string):
    denomination_values = string.split(",")
    return DenominationSet([int(a) for a in denomination_values])
    
coin_set_input = widgets.Text(
    value='1,1,1',
    description='Covering Coin Set',
)
denomination_set_input = widgets.Text(
    value='4,2,1',
    description='Denomination',
)
button = widgets.Button(description="Get Reaching Coin Sets")
maximum_input = widgets.BoundedIntText(
    value=3,
    min=0,
    max=100,
    step=1,
    description='Maximum:',
    disabled=False
)

def display_reaching_coin_sets(a):
    maximum = maximum_input.value
    denomination = string_to_denomination(denomination_set_input.value)
    if coin_set_input.value != "":
        
        covering_coin_set = string_to_coin_set(coin_set_input.value, denomination)
    else:
        covering_coin_set = evaluate_denomination_naive(maximum, denomination, allow_vestigal = False)[0]
    data = []
    for i in range(maximum):
        reaching_coin_set = get_reaching_coin_set(covering_coin_set, i)
        new_data = {}
        new_data['total'] = i
        for j in range(len(reaching_coin_set.denomination_set)):
            new_data[str(reaching_coin_set.denomination_set[j])] = reaching_coin_set[j]
        data.append(new_data)
    table = tabulate.tabulate(data, tablefmt='html', headers='keys')
    display(table)

button.on_click(display_reaching_coin_sets)
        
display(coin_set_input)
display(denomination_set_input)
display(maximum_input)
display(button)

Text(value='1,1,1', description='Covering Coin Set')

Text(value='4,2,1', description='Denomination')

BoundedIntText(value=3, description='Maximum:')

Button(description='Get Reaching Coin Sets', style=ButtonStyle())

## Observations from Naive Attempt

### On verifying and generating covering sets

I think, but have not proven, that there is a relationship that holds for all covering coin sets. This relationship is given by:

$$d_{n}\leq(\sum_{n-1}^{i=1}d_{i}c_{i})+1$$

If we rearrange these terms, we can solve for $c_{n}$

$$c_{n}\geq\frac{d_{n+1}-(\sum_{n-1}^{i=1} d_{i}c_{i})-1}{d_{n}}$$

In [10]:
def is_covering_coin_set(coin_set: CoinSet, maximum: int):
    running_sum = 0
    for n in range(len(coin_set.denomination_set)):
        d_n1 = 0
        if (n == len(coin_set.denomination_set)-1):
            d_n1 = maximum
        else:
            d_n1 = coin_set.denomination_set[len(coin_set.denomination_set)- (n+1) -1]
        d_n = coin_set.denomination_set[len(coin_set.denomination_set)-n -1]
        c_n = coin_set[len(coin_set.denomination_set) - n -1]
        if c_n < ((d_n1 - running_sum -1)/(d_n)):
            return False
        running_sum += d_n*c_n
    return True

d = DenominationSet([6,3,1])
print(is_covering_coin_set(CoinSet([1,1,1], d), 10))
print(is_covering_coin_set(CoinSet([1,1,3], d), 10))

False
True


To test this relationship, we can leverage the naive solution and compare.

In [11]:
def test_covering_coin_set_evaluator(maximum):
    for d in get_all_possible_denominations(maximum):
        for x in range(maximum*2):
            solutions = []
            done = False
            for a in get_all_permutations_summing_to_x(x, len(d)):
                coins = CoinSet(a, d)
                can_reach_all = True
                for i in range(maximum):
                    if not can_coin_set_reach_total(coins, i):
                        can_reach_all = False
                        break
                assert can_reach_all == is_covering_coin_set(coins, maximum)
                if (can_reach_all):
                    done = True
            if done:
                break
perform_test_button = widgets.Button(description="Perform Test")
maximum_input = widgets.BoundedIntText(
    value=3,
    min=0,
    max=100,
    step=1,
    description='Maximum:',
    disabled=False
)

def run_test(a):
    test_covering_coin_set_evaluator(maximum_input.value)
    print("Done!")

perform_test_button.on_click(run_test)
    
display(maximum_input)
display(perform_test_button)

BoundedIntText(value=3, description='Maximum:')

Button(description='Perform Test', style=ButtonStyle())

It sure seems like it holds.

If it holds, we can actually generate covering coin sets by setting the greater than to equals! Ergo, we can skip searching for a covering coin set...

In [12]:
def get_minimum_covering_coin_set(d : DenominationSet, maximum: int, allow_vestigal = False) -> CoinSet:
    running_sum = 0
    coins = []
    for n in range(len(d)):
        d_n1 = 0
        if (n == len(d)-1):
            d_n1 = maximum
        else:
            d_n1 = d[len(d)- (n+1) -1]
        d_n = d[len(d)-n -1]
        c_n = ceil((d_n1 - running_sum -1)/(d_n))
        if not allow_vestigal and c_n <= 0:
            c_n = 1
        coins = [c_n] + coins
        running_sum += d_n*c_n
    return CoinSet(coins, d)

print(get_minimum_covering_coin_set(DenominationSet([5, 2, 1]), 10))
print(get_minimum_covering_coin_set(DenominationSet([8,3,2,1]), 10))

[(1*5), (2*2), (1*1)]
[(1*8), (2*3), (1*2), (1*1)]


The unfortunate thing is that it only returns one covering coin set, as opposed to the several that may exist. For example, for [5, 2, 1], there are 2 covering coin sets, [1, 2, 1] and [1, 1, 2]. I'm sure that I can figure out why this is eventually but ATM there is no theory for it. TODO.

Despite this, I went ahead and added this as an optional toggle. This improvement sped up the computation speed by 10!

### On generating denominations

Possibly more importantly, we can use this equation to generate denominations that fit a covering coin set, allowing us to effectively go backwards: Start with a covering coin set, and derive what the denomination must be! We simply make the greater than or equal to just an equals...

In [13]:
def get_prime_denomination(covering_coin_set: list[int], tabular_mode = False) -> tuple[DenominationSet, int]:
    running_sum = 0
    denomination = []
    if tabular_mode:
        data = []
    for i in range(len(covering_coin_set)):
        if i == 0:
            d_n = 1
        else:
            d_n = running_sum + 1
        denomination = [d_n] + denomination
        c_n = covering_coin_set[len(covering_coin_set)-i-1]
        running_sum+=c_n*d_n
        if tabular_mode:
            data.append({
                'Dn' : d_n,
                'Cn' : c_n,
                "Sigma": running_sum
            })
    if (tabular_mode):
        return data
    else:
        return (DenominationSet(denomination), running_sum+1)

print(get_prime_denomination([1,1,2]))

([6, 3, 1], 12)


In [14]:
coin_set_input = widgets.Text(
    value='1,1,1',
    description='Covering Coin Set',
)
button = widgets.Button(description = "Generate")

def output_table(a):
    table = tabulate.tabulate(get_prime_denomination([int(i) for i in coin_set_input.value.split(",")], tabular_mode = True), tablefmt='html', headers='keys')
    display(table)
    
button.on_click(output_table)

display(coin_set_input)
display(button)

Text(value='1,1,1', description='Covering Coin Set')

Button(description='Generate', style=ButtonStyle())

Now, the problem is that this only outputs one denomination. I call this denomination the **prime denomination** of a covering coin set.

Furthermore, you'll notice that this function does not rely on the maximum value at all. Indeed, the maximum value representable by a covering coin set is dependent only upon the coin set itself, not the denomination. The maximum value that it is able to represent is conveinently given by the total cost of the covering coin set, which is computed as a by-product of determining the denomination. I call this number the covering coin set's **power**.

What about the other, **non-prime** denominations? These occur within the context of a maximum. If the prime denomination's power is greater than the maximum, then there *may* be non-prime denominations. This is due to the fact that we solved for the prime denomination by only considering the "equality" case, while "less than" is still an option.

$$\alpha=\frac{prime-maximum}{c_{n}}$$

$ \alpha $ represents the maximum amount "n" can decrease while still working for the maximum. Of course, we require that

$$\alpha<d_{n}-d_{n-1}$$

...or else we would go below the previous denomination! TODO: This doesn't 100% work

In [15]:
def get_all_denominations_for_covering_coin_set(covering_coin_set: list[int], maximum : int) -> list[DenominationSet]:
    prime, power = get_prime_denomination(covering_coin_set)
        
    if (power < maximum):
        return []
    elif (power == maximum):
        return [prime]
    
    #First element is the incomplete denomination, second is the remaining power
    incomplete_denominations = [([], power)]
    for i in range(len(covering_coin_set)):
        if i == len(covering_coin_set)-1:
            continue
        d_n = prime[i]
        d_n1 = prime[i+1]
        c_n = covering_coin_set[i]
        print(f"i {i} d_n {d_n} d_n1 {d_n1} c_n {c_n}")
        new_incomplete_denominations = []
        for incomplete_denomination in incomplete_denominations:
            alpha = min(int((incomplete_denomination[1]-maximum)/c_n), d_n - d_n1)
            print(f"\tincomplete {incomplete_denomination} alpha_a {(incomplete_denomination[1]-maximum)/c_n} alpha_b {d_n - d_n1} alpha {alpha}")
            if alpha <= 0:
                print(f"\t\tCan do nothing else here. Appending {d_n}")
                new_incomplete_denominations.append((incomplete_denomination[0] + [d_n], incomplete_denomination[1]))
            else:
                for a in range(alpha):
                    new_denomination = d_n-a
                    new_power = incomplete_denomination[1]-a*c_n
                    print(f"\t\tAppending {new_denomination}, Power = {new_power}")
                    new_incomplete_denominations.append((incomplete_denomination[0] + [new_denomination], new_power))
        incomplete_denominations = new_incomplete_denominations
    return [DenominationSet([i[0]+[1]]) for i in incomplete_denominations]

print(get_all_denominations_for_covering_coin_set([1,1,1,1], 10))

i 0 d_n 8 d_n1 4 c_n 1
	incomplete ([], 16) alpha_a 6.0 alpha_b 4 alpha 4
		Appending 8, Power = 16
		Appending 7, Power = 15
		Appending 6, Power = 14
		Appending 5, Power = 13
i 1 d_n 4 d_n1 2 c_n 1
	incomplete ([8], 16) alpha_a 6.0 alpha_b 2 alpha 2
		Appending 4, Power = 16
		Appending 3, Power = 15
	incomplete ([7], 15) alpha_a 5.0 alpha_b 2 alpha 2
		Appending 4, Power = 15
		Appending 3, Power = 14
	incomplete ([6], 14) alpha_a 4.0 alpha_b 2 alpha 2
		Appending 4, Power = 14
		Appending 3, Power = 13
	incomplete ([5], 13) alpha_a 3.0 alpha_b 2 alpha 2
		Appending 4, Power = 13
		Appending 3, Power = 12
i 2 d_n 2 d_n1 1 c_n 1
	incomplete ([8, 4], 16) alpha_a 6.0 alpha_b 1 alpha 1
		Appending 2, Power = 16
	incomplete ([8, 3], 15) alpha_a 5.0 alpha_b 1 alpha 1
		Appending 2, Power = 15
	incomplete ([7, 4], 15) alpha_a 5.0 alpha_b 1 alpha 1
		Appending 2, Power = 15
	incomplete ([7, 3], 14) alpha_a 4.0 alpha_b 1 alpha 1
		Appending 2, Power = 14
	incomplete ([6, 4], 14) alpha_a 4.0

So, given that the poewr of a covering coin set is only depedent on the coin set, the question is can we get a form of the equation that does not rely on denominations at all? Of course! There is a fascinating relationship I just discovered that could crack this case wide open

$$ d_{n}=(\sum_{n-1}^{i=1}d_{i}c_{i})+1 $$

$$d_{1}=1 $$

$$d_{2}=d_{1}c_{1}+1$$

$$=c_{1}+1$$

$$d_{3}=d_{2}c_{2}+d_{1}c_{1}+1$$

$$=(c_{1}+1)c_{2}+c_{1}+1$$

$$=(c_{1}+1)(c_{2}+1)$$

That is,

$$d_{n}=\prod_{i=1}^{n-1}c_{i}+1$$

The interesting thing is that this means that d_n will be the same regardless of the order of the predecssor coins! Let's test this theory

In [16]:
def fast_computation_of_dn(coins):
    a = 1
    
    for i in coins[1:]:
        a*=(i+1)
    return a

def demonstrate_rearrangement(coins):
    print(get_prime_denomination(coins)[0], fast_computation_of_dn(coins))

    
demonstrate_rearrangement([1,2,3,4])
demonstrate_rearrangement([1,3,2,4])
demonstrate_rearrangement([1,4,3,2])
demonstrate_rearrangement([1,4,2,3])
demonstrate_rearrangement([1,2,4,3])


[60, 20, 5, 1] 60
[60, 15, 5, 1] 60
[60, 12, 3, 1] 60
[60, 12, 4, 1] 60
[60, 20, 4, 1] 60


(Oh, also, the power of the covering coin set is just d_n+1)